In [1]:
import pandas as pd 
import numpy as np 
import re
import zipfile #used to import the data from the zip file 

In [2]:
df_zip = zipfile.ZipFile('C:/Users/yesha/Downloads/movies_dataset.csv.zip')
df = pd.read_csv(df_zip.open('movies_dataset.csv'))
df.head()

,Unnamed: 0,IMDb-rating,appropriate_for,director,downloads,id,industry,language,posted_date,release_date,run_time,storyline,title,views,writer
0,0,4.8,R,John Swab,304,372092,Hollywood / English,English,"20 Feb, 2023",Jan 28 2023,105,Doc\r\n facilitates a fragile truce between th...,Little Dixie,"2,794",John Swab
1,1,6.4,TV-PG,Paul Ziller,73,372091,Hollywood / English,English,"20 Feb, 2023",Feb 05 2023,84,Caterer\r\n Goldy Berry reunites with detectiv...,Grilling Season: A Curious Caterer Mystery,"1,002",John Christian Plummer
2,2,5.2,R,Ben Wheatley,"1,427",343381,Hollywood / English,"English,Hindi","20 Apr, 2021",Jun 18 2021,1h 47min,As the world searches for a cure to a disastro...,In the Earth,"14,419",Ben Wheatley
3,3,8.1,NaN,Venky Atluri,"1,549",372090,Tollywood,Hindi,"20 Feb, 2023",Feb 17 2023,139,The life of a young man and his struggles agai...,Vaathi,"4,878",Venky Atluri
4,4,4.6,NaN,Shaji Kailas,657,372089,Tollywood,Hindi,"20 Feb, 2023",Jan 26 2023,122,A man named Kalidas gets stranded due to the p...,Alone,"2,438",Rajesh Jayaraman


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20548 entries, 0 to 20547
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       20548 non-null  int64  
 1   IMDb-rating      19707 non-null  float64
 2   appropriate_for  11072 non-null  object 
 3   director         18610 non-null  object 
 4   downloads        20547 non-null  object 
 5   id               20548 non-null  int64  
 6   industry         20547 non-null  object 
 7   language         20006 non-null  object 
 8   posted_date      20547 non-null  object 
 9   release_date     20547 non-null  object 
 10  run_time         18780 non-null  object 
 11  storyline        18847 non-null  object 
 12  title            20547 non-null  object 
 13  views            20547 non-null  object 
 14  writer           18356 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 2.4+ MB


In [4]:
print(df.dtypes)

Unnamed: 0           int64
IMDb-rating        float64
appropriate_for     object
director            object
downloads           object
id                   int64
industry            object
language            object
posted_date         object
release_date        object
run_time            object
storyline           object
title               object
views               object
writer              object
dtype: object


### Data Cleaning

'run_time' has a mixed format, with some in the form of hours and minutes and others in just minutes 

We also converted 'views', 'downloads' and 'run_time' to a numeric data type, and deleted unecessary additions like the coma 

In [5]:
# Define a regular expression to match values in the format Xh Ymin or Xmin
pattern = re.compile(r'^(\d+)(h\s)?(\d+)?(min)?$')

# Convert run_time into a string 
df['run_time'] = df['run_time'].map(str)

# Apply the conversion code only to values that match the pattern
def convert_runtime(value):
    if pattern.match(value):
        value = value.replace('h', ' ').replace('min', '')
        split_value = value.split()
        if len(split_value) >= 2:
            return pd.to_numeric(split_value[0])*60 + pd.to_numeric(split_value[1])
        else:
            return pd.to_numeric(split_value[0])
    else:
        return value
    
# Update the runtime values   
df['run_time'] = df['run_time'].apply(convert_runtime)

# Convert data types and delete unecessary additions 
df['views'] = pd.to_numeric(df['views'].str.replace(',', ''))
df['downloads'] = pd.to_numeric(df['downloads'].str.replace(',', ''))
df['run_time'] = pd.to_numeric(df['run_time'], errors='coerce')

# Dropping repeated data values 
df = df.drop_duplicates(subset='title')

### Null values
There are some null values from the dataset, hence we need to analyse the data and fill in the missing values with either the mean or median, whichever more suitable.

In [6]:
# Check the number of null values 
df.isnull().sum()

Unnamed: 0            0
IMDb-rating         473
appropriate_for    7095
director           1521
downloads             1
id                    0
industry              1
language            526
posted_date           1
release_date          1
run_time           1673
storyline          1271
title                 1
views                 1
writer             1770
dtype: int64

From this information we can see that 'appropriate_for' has the largest number of null values, almost 50% of the entire data. <b> Hence, we decided to drop the variable. (tbc) <b> 

For numerical data such as 'downloads', 'run_time' and 'views', the missing values can be filled in using the mean or the median.

However for the rest of the data, each data entry is independent from other data entries (eg the storyline), making it hard to fill in the missing values accurately. Hence for these data we decided to drop the missing values. 

### Filling in the missing values 

For the numeric data types, we can choose to fill in the missing values with either the mean or media. To determine the better choice, we look at the number of outliers. 

In [7]:
# Create separate datadrames 
numeric_data = df.filter(['downloads', 'run_time', 'views', 'IMDb-rating'])
numeric_data.head()

,downloads,run_time,views,IMDb-rating
0,304.0,105.0,2794.0,4.8
1,73.0,84.0,1002.0,6.4
2,1427.0,107.0,14419.0,5.2
3,1549.0,139.0,4878.0,8.1
4,657.0,122.0,2438.0,4.6


In [8]:
# Lets learn more about the statistics of our numeric data 
numeric_data.describe()

,downloads,run_time,views,IMDb-rating
count,16572.000000,14900.000000,1.657200e+04,16100.000000
mean,12560.936097,106.191342,4.111690e+04,5.598975
std,25594.765487,24.458805,6.711704e+04,1.281189
min,0.000000,2.000000,6.670000e+02,1.100000
25%,1265.000000,90.000000,9.811250e+03,4.700000
50%,3708.000000,99.000000,1.913650e+04,5.700000
75%,12202.250000,118.000000,4.384750e+04,6.500000
max,391272.000000,321.000000,1.638533e+06,9.900000


In [9]:
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

In [10]:
print("No. of outliers in columns: \n")
((numeric_data < (Q1-1.5*IQR)) | (numeric_data > (Q3+1.5*IQR))).sum()

No. of outliers in columns: 



downloads      1850
run_time        584
views          1617
IMDb-rating      51
dtype: int64

Given the larger number of outliers for 'downloads', 'views' and 'run_time', we will use median to fill in the missing values.
For 'IMDb-rating', we use mean.

In [11]:
# Filling in 'downloads', 'views' and 'run_time' with median 
df['downloads'] = df['downloads'].fillna(df['downloads'].median())
df['views'] = df['views'].fillna(df['views'].median())
df['run_time'] = df['run_time'].fillna(df['run_time'].median())

# Fill in 'IMDb-rating' with mean 
df['IMDb-rating'] = df['IMDb-rating'].fillna(df['IMDb-rating'].mean())

In [12]:
# Drop the rest of the missing values 
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8429 entries, 0 to 20533
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       8429 non-null   int64  
 1   IMDb-rating      8429 non-null   float64
 2   appropriate_for  8429 non-null   object 
 3   director         8429 non-null   object 
 4   downloads        8429 non-null   float64
 5   id               8429 non-null   int64  
 6   industry         8429 non-null   object 
 7   language         8429 non-null   object 
 8   posted_date      8429 non-null   object 
 9   release_date     8429 non-null   object 
 10  run_time         8429 non-null   float64
 11  storyline        8429 non-null   object 
 12  title            8429 non-null   object 
 13  views            8429 non-null   float64
 14  writer           8429 non-null   object 
dtypes: float64(4), int64(2), object(9)
memory usage: 1.0+ MB


In [13]:
df

,Unnamed: 0,IMDb-rating,appropriate_for,director,downloads,id,industry,language,posted_date,release_date,run_time,storyline,title,views,writer
0,0,4.8,R,John Swab,304.0,372092,Hollywood / English,English,"20 Feb, 2023",Jan 28 2023,105.0,Doc\r\n facilitates a fragile truce between th...,Little Dixie,2794.0,John Swab
1,1,6.4,TV-PG,Paul Ziller,73.0,372091,Hollywood / English,English,"20 Feb, 2023",Feb 05 2023,84.0,Caterer\r\n Goldy Berry reunites with detectiv...,Grilling Season: A Curious Caterer Mystery,1002.0,John Christian Plummer
2,2,5.2,R,Ben Wheatley,1427.0,343381,Hollywood / English,"English,Hindi","20 Apr, 2021",Jun 18 2021,107.0,As the world searches for a cure to a disastro...,In the Earth,14419.0,Ben Wheatley
7,7,6.5,R,Benjamin Caron,1781.0,371751,Hollywood / English,English,"13 Feb, 2023",Feb 17 2023,116.0,"Motivations are suspect, and expectations are ...",Sharper,18225.0,"Brian Gatewood, Alessandro Tanaka"
8,8,6.9,PG-13,Ravi Kapoor,458.0,372042,Hollywood / English,English,"18 Feb, 2023",Dec 02 2022,80.0,An\r\n unmotivated South Asian American rapper...,Four Samosas,6912.0,Ravi Kapoor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20496,20496,7.6,Approved,"William Cottrell, David Hand, Wilfred Jackson",41145.0,920,Anime / Kids,English,"01 Jan, 1970",Feb 03 1938,83.0,"The first, and by far most memorable full-leng...",Snow White and the Seven Dwarfs,66860.0,"Jacob Grimm, Wilhelm Grimm, Ted Sears"
20497,20497,3.3,R,"Sam Gorski, Niko Pueringer",9551.0,921,Hollywood / English,English,"01 Jan, 1970",Jan 01 1970,87.0,After losing contact with its researchers on a...,Dark Island,25581.0,"Simon Boyes, Gregory Gieras, Patrick Ewald"
20499,20499,3.6,Not Rated,Keith Alan Morris,8953.0,994,Hollywood / English,English,"01 Jan, 1970",Jan 01 1970,98.0,Gutter King is an urban coming-of-age fight dr...,Gutter King,33431.0,Keith Alan Morris
20522,20522,7.1,Not Rated,Biren Nag,1932.0,23825,Bollywood / Indian,Hindi,"01 Jan, 1970",May 11 1962,158.0,"After a lusty Thakur rapes a young girl, she k...",Bees Saal Baad,6076.0,"Dhruva Chatterjee, Dev Kishan"


In [27]:
df.to_csv('cleaned-df3.csv')